# 评测chatgpt解决指代消解问题

In [1]:
import os
from typing import List

from datasets import load_dataset



In [2]:
wsc273_dataset = load_dataset('winograd_wsc', name='wsc273', split='test')
wsc273_dataset

Dataset({
    features: ['text', 'pronoun', 'pronoun_loc', 'quote', 'quote_loc', 'options', 'label', 'source'],
    num_rows: 273
})

In [ ]:
!pip install openai==0.28.0

In [14]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

In [ ]:
openai.__version__

In [15]:
def query_format_helper(pronoun: str, answers: List[str]) -> str:
    return f'Q: In the previous statement, does "{pronoun}" refer to {answers[0]} or {answers[1]}? A:'


def construct_query_from_schema(
        text: str, pronoun: str, answers: List[str]
) -> str:
    return f"S:{text} {query_format_helper(pronoun, answers)}"

c = 0
for example in wsc273_dataset:
    query = construct_query_from_schema(example['text'], example['pronoun'], example['options'])
    print(query)
    break

S:The city councilmen refused the demonstrators a permit because they feared violence. Q: In the previous statement, does "they" refer to The city councilmen or The demonstrators? A:


In [16]:
def get_openai_answer(query_prompt: str, add_leading: bool = True) -> str:
    init_prompt_starter = "The following are pairs of Winograd Schema in the form of a statement S, a question Q, and an answer A:"
    init_prompt1 = "S: The cat went through the door, but it's tail got stuck. Q: In the previous statement, what does 'it' refer to? A: The cat."
    init_prompt2 = "S: The cat tried to go through the door, but it was too small. Q: In the previous statement, what does 'it' refer to? A: The door."
    init_prompt3 = "S: Fedex made more profit than UPS last year, but that was mostly due to the success of the new delivery system they implemented. Q: In the previous statement, what does 'they' refer to? A: Fedex."
    init_prompt4 = "S: Sam tried to buy Xerxes lunch, but he wouldn't allow it. Q: In the previous statement, who does 'he' refer to? A: Xerxes."

    # add leading prompts to cue model...
    if add_leading:
        query_prompt = f"{init_prompt_starter} {init_prompt1}, {init_prompt2}, {init_prompt3}, {init_prompt4}, {query_prompt}"

    response = openai.ChatCompletion.create(
        model="gpt-4",
        prompt=query_prompt,
        temperature=0,
        max_tokens=200,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=["\n"]
    )

    return response['choices'][0]['text']

In [17]:
c = 0
for example in wsc273_dataset:
    query = construct_query_from_schema(example['text'], example['pronoun'], example['options'])
    print(query)
    r = get_openai_answer(query)
    print(r)
    break

S:The city councilmen refused the demonstrators a permit because they feared violence. Q: In the previous statement, does "they" refer to The city councilmen or The demonstrators? A:


APIError: HTTP code 200 from API (<!doctype html><html lang="en"><head><meta charset="utf-8"/><link rel="icon" href="logo.png"/><meta name="viewport" content="width=device-width,initial-scale=1"/><meta name="theme-color" content="#ffffff"/><meta name="description" content="OpenAI 接口聚合管理，支持多种渠道包括 Azure，可用于二次分发管理 key，仅单可执行文件，已打包好 Docker 镜像，一键部署，开箱即用"/><title>One API</title><script defer="defer" src="/static/js/main.d26af7bf.js"></script><link href="/static/css/main.54631fc9.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div></body></html>)